#### This code start from the raw recordings and separate data into the classes of the dataset

In [1]:
import pandas as pd
import numpy as np
import os
from utils.loader import load_data, unicorn_fs

path_recording = 'data/recordings/recordings_04_04_2024_OSCAR'
unicorn_channels = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "acc_x", "acc_y", "acc_z"]
path_dataset = 'data/dataset/dataset_04_04_2024_OSCAR'

#### RECORDINGS 04-04-2024 - OSCAR
4 EEG and accellerometers recordings all the possible combinations of:

- recording type: [listening, playing_listening]
- song types (labels): [relax, excited]

A recording is done as the following:

1. 5 seconds to stabilize the signal 
2. 30 second baseline
3. song 1
4. 30 second baseline
5. song 2
6. 30 second baseline
7. song 3
8. 30 second baseline
9. song 4

total: 6 minutes and 5 seconds per recording

In [12]:
recording_type = ['listening', 'playing_listening']
music_type = ['relax', 'excited']

for rec_type in recording_type:
    for mus_type in music_type:
        
        # Path
        path_data = os.path.join(path_recording, rec_type, mus_type)
        files = [f for f in os.listdir(path_data) if f.endswith('.csv')]
        path_file = os.path.join(path_data, files[0])

        # Load data
        skip_seconds = 5
        eeg, trigger, dataframe = load_data(path_file, header=False, fs=unicorn_fs, skiprows=skip_seconds, names = unicorn_channels)

        # Duration in minutes and seconds of the recording
        duration = eeg.shape[0] / unicorn_fs
        minutes = int(duration / 60)
        seconds = int(duration % 60)
        print(f'The recording is {minutes} minutes and {seconds} seconds long')

        print(f'Processing file: {path_file}')

        # 30 seconds silence
        silence1 = eeg[:unicorn_fs*30, :]
        # 1 minute listening song_1
        song1 = eeg[unicorn_fs*30:unicorn_fs*90, :]
        # 30 seconds silence
        silence2 = eeg[unicorn_fs*90:unicorn_fs*120, :]
        # 1 minute listening song_2
        song2 = eeg[unicorn_fs*120:unicorn_fs*180, :]
        # 30 seconds silence
        silence3 = eeg[unicorn_fs*180:unicorn_fs*210, :]
        # 1 minute listening song_3
        song3 = eeg[unicorn_fs*210:unicorn_fs*270, :]
        # 30 seconds silence
        silence4 = eeg[unicorn_fs*270:unicorn_fs*300, :]
        # 1 minute listening song_4
        song4 = eeg[unicorn_fs*300:unicorn_fs*360, :]

        # concatenate silences and songs
        silence = np.concatenate((silence1, silence2, silence3, silence4), axis=0)
        songs = np.concatenate((song1, song2, song3, song4), axis=0)

        # convert to dataframes
        silence_df = pd.DataFrame(silence)
        songs_df = pd.DataFrame(songs)

        # save dataframes
        baseline_file_name = 'baseline_' + rec_type + '_' + mus_type + '.csv'
        signal_file_name = 'song_' + rec_type + '_' + mus_type + '.csv'

        path_baseline_folder = os.path.join(path_dataset, rec_type, 'baseline')
        path_signal_folder = os.path.join(path_dataset, rec_type, mus_type)

        if not os.path.exists(path_baseline_folder):
            os.makedirs(path_baseline_folder)
        if not os.path.exists(path_signal_folder):
            os.makedirs(path_signal_folder)

        path_baseline = os.path.join(path_baseline_folder, baseline_file_name)
        path_signal = os.path.join(path_signal_folder, signal_file_name)

        silence_df.to_csv(path_baseline, index=False, header=False)
        songs_df.to_csv(path_signal, index=False, header=False)

        print(f'Files saved in:\t{path_baseline}\n\t\t{path_signal}\n')

The recording is 6 minutes and 6 seconds long
Processing file: data/recordings/recordings_04_04_2024_OSCAR\listening\relax\relax_04_04_2024_16_19_17.csv
Files saved in:	data/dataset/dataset_04_04_2024_OSCAR\listening\baseline\baseline_listening_relax.csv
		data/dataset/dataset_04_04_2024_OSCAR\listening\relax\song_listening_relax.csv

The recording is 6 minutes and 7 seconds long
Processing file: data/recordings/recordings_04_04_2024_OSCAR\listening\excited\excited_04_04_2024_16_27_52.csv
Files saved in:	data/dataset/dataset_04_04_2024_OSCAR\listening\baseline\baseline_listening_excited.csv
		data/dataset/dataset_04_04_2024_OSCAR\listening\excited\song_listening_excited.csv

The recording is 6 minutes and 6 seconds long
Processing file: data/recordings/recordings_04_04_2024_OSCAR\playing_listening\relax\relax_04_04_2024_16_36_35.csv
Files saved in:	data/dataset/dataset_04_04_2024_OSCAR\playing_listening\baseline\baseline_playing_listening_relax.csv
		data/dataset/dataset_04_04_2024_OSC